In [1]:
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pytz

import importlib.util
import sys


spec = importlib.util.spec_from_file_location("carbon", "../src/carbon.py")
foo = importlib.util.module_from_spec(spec)
sys.modules["carbon"] = foo
spec.loader.exec_module(foo)

import carbon


traces = ['phased'] # , "pai_200"

waiting_times = ["48", "6x24", "4"]

carbon_trace = carbon.get_carbon_model("AU-SA", 17544+7000, extra_columns=False).reindex(0)
# carbon_trace = carbon.get_carbon_model("DE-hourly-start-july", 0, extra_columns=False)


In [2]:
import sys
sys.path.append('../src/')

from scheduling.suspend_phases_scheduling_policy import SchedulerDebugOptions

import scheduling as gaia_sched
import carbon
import power_consumption_profiles as pcp

# scheduler = gaia_sched.SuspendSchedulingDynamicPowerPolicy(None, carbon_trace)
# options = SchedulerDebugOptions(
#         use_startup=True, 
#         dynamic_power=True, 
#         use_progress=True, 
#         linearize=True,
#         timelimit=None,
#         scale_time=True,
# )
# 
# foo_model = pcp.PeriodicPowerFunction({
#     'startup':[{'name': 'startup','duration': 0, 'power': 150}],
#     'work':[{'name': 'high', 'power': 200, 'duration': 3600}, {'name': 'low', 'power': 100, 'duration': 3600}]
# }, length=3*60*30)
# 
# test_model = pcp.PeriodicPowerFunction({
#     'startup':[{'name': 'startup','duration': 0, 'power': 200}],
#     'work':[{'name': 'high', 'power': 200, 'duration': 1800}, {'name': 'low', 'power': 100, 'duration': 3600}]
# }, length=9600)
# 
# debug_result = scheduler.find_execution_times(carbon_trace, 4*60*60, test_model, debugOptions=options)
# real_result = scheduler.find_execution_times(carbon_trace, 4*60*60, test_model)

In [3]:
# use this cell to create the plot results
shared_deadline = 250

scheduler = gaia_sched.SuspendSchedulingDynamicPowerPolicy(None, carbon_trace)
options_1 = SchedulerDebugOptions(
        use_startup=False, 
        dynamic_power=False, 
        use_progress=False, 
        linearize=False,
        timelimit=None,
        scale_time=False,
)

job_no_startup = pcp.PowerFunction({
    'startup': [ ],
    'work': [
      {'name': 'Constant', 'duration': 100, 'power': 100}, 
    ]
})

result_1 = scheduler.find_execution_times(carbon_trace, shared_deadline, job_no_startup, options_1)

# options_2 = SchedulerDebugOptions(
#         use_startup=True, 
#         dynamic_power=False, 
#         use_progress=False, 
#         linearize=False,
#         timelimit=None,
#         scale_time=False,
# )

constant_job = pcp.PowerFunction({
    'startup': [
        {'name': 'Constant', 'duration': 20, 'power': 100}
    ],
    'work': [
        {'name': 'Constant', 'duration': 100, 'power': 100}, 
    ]
})


# There was some bug when instrumenting the scheduler,
# it worked earlier I swear
# result_2 = scheduler.find_execution_times(carbon_trace, shared_deadline, constant_job, options_2)

options_3 = SchedulerDebugOptions(
        use_startup=True, 
        dynamic_power=False, 
        use_progress=True, 
        linearize=False,
        timelimit=None,
        scale_time=False,
)


job = pcp.PowerFunction({
    'startup': [
        {'name': 'Start Python', 'duration': 10, 'power': 50},
        {'name': 'Download Data', 'duration': 10, 'power': 70},
    ],
    'work': [
      {'name': 'High', 'duration': 50, 'power': 230.0}, 
      {'name': 'Low', 'duration': 50, 'power': 100}, 
    ]
})

result_3 = scheduler.find_execution_times(carbon_trace, shared_deadline, constant_job, options_3)

options_4 = SchedulerDebugOptions(
        use_startup=True, 
        dynamic_power=True, 
        use_progress=True, 
        linearize=True,
        timelimit=None,
        scale_time=False,
)

result_4 = scheduler.find_execution_times(carbon_trace, shared_deadline, job, options_4)

scaled deadlines is: 250, spt: 1
Set parameter Username
Set parameter LogFile to value "gurobi.log"
Set parameter Threads to value 4
Using license file /opt/gurobi1103/gurobi.lic
Academic license - for non-commercial use only - expires 2025-08-12

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 23.10")
Copyright (c) 2024, Gurobi Optimization, LLC

Read LP format model from file /tmp/0b4bdab105e1418ea110f5fc7447b653-pulp.lp
Reading time = 0.01 seconds
OBJ: 4 rows, 501 columns, 502 nonzeros

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4 rows, 501 columns and 502 nonzeros
Model fingerprint: 0x17395340
Variable types: 1 continuous, 500 integer (500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 4e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 1e+02]
Found heuristic sol

In [10]:

from typing import List
import plotly.io as pio
import pulp

color_map = {
    "Work": "red",
    "Starting": "green",
    "Startup finished": "lime",
    "Idle": "black"
}

shared_height = 250
shared_width = 1020

def add_common_styling_to_fig(fig):
    fig.update_layout({
        'margin': dict(l=10, r=10, t=10, b=10)
    })


def find_upper_lower_index_for_phases(bool_list: List[bool]):
    groups = []
    start_index = None

    for i, value in enumerate(bool_list):
        if value: 
            if start_index is None:
                start_index = i
        else:
            if start_index is not None:
                groups.append((start_index, i - 1)) 
                start_index = None

    if start_index is not None:
        groups.append((start_index, len(bool_list) - 1))

    # Output the result
    return groups

def make_carbon_trace_with_work_or_startup(targets: List[tuple]):
    # add the work or startup information to the carbon_trace

    graph_times = [i for i in range (shared_deadline)]
    graph_carbon_costs = carbon_trace.df["carbon_intensity_avg"].head(shared_deadline)
    category = ['Idle'] * shared_deadline

    for variable, name in targets:
        for t in range(len(variable)):
            if pulp.value(variable[t]):
                category[t] = name

    fig_carbon = go.Figure()

    fig_carbon.add_trace(go.Scatter(x=graph_times, y=graph_carbon_costs, mode='lines', showlegend=False, line=dict(color='black', width=2)))

    marker_fig = px.scatter(x=graph_times, y=graph_carbon_costs, color=category, color_discrete_map={
                "Work": "red",
                "Starting": "green",
                "Startup finished": "lime",
                "Idle": "black"}
    )

    for trace in marker_fig.data:
        fig_carbon.add_trace(trace)


    fig_carbon.update_layout({ 
        'yaxis_title': 'Carbon intensity in kgCO2/kWh', 
        'xaxis_title': 'Time slots'
    })
    
    return fig_carbon


def make_trace_from_variables(targets: List[tuple]):
    fig = go.Figure()

    for variable, name in targets:
        times = [t for t in range(len(variable))]
        values = [pulp.value(variable[t]) if pulp.value(variable[t]) is not None else None for t in range(len(variable))]

        to_remove = [i for i, value in enumerate(values) if value == 0.0]

        filtered_times = [value for index, value in enumerate(times) if index not in to_remove]
        filtered_values = [value for index, value in enumerate(values) if index not in to_remove]
        # the things you do for a cool plot

        fig.add_trace(go.Scatter(x=filtered_times, y=filtered_values, showlegend=False, mode='markers',
        marker=dict(
            color=color_map[name],
        )))

    return fig

common_styling = {
    'xaxis_title': 'Time slots'
}

def first_plot_work_only():
    """first plot should just show the timeslot were work is being done
    """

    carbon_plot = make_carbon_trace_with_work_or_startup([(result_1["work"], 'Work')])

    carbon_plot.update_layout(height=shared_height, width=shared_width)

    add_common_styling_to_fig(carbon_plot)

    carbon_plot.update_layout(legend ={
        "yanchor":"top",
        "y":0.97,
        "xanchor":"left",
        "x":0.01
    })

    carbon_plot.show()

    pio.write_image(carbon_plot, file="lp_work.pdf", format='pdf')

    return carbon_plot

def second_plot_overhead_via_startup():
    """second plot includes overhead for every start
    """

    carbon_plot = make_carbon_trace_with_work_or_startup([(result_3["work"], 'Work'), (result_3["starting"], 'Starting'), (result_3["startup_finished"], 'Startup finished')])
    add_common_styling_to_fig(carbon_plot)
    carbon_plot.update_layout(height=shared_height, width=shared_width)
    carbon_plot.update_layout(legend ={
        "yanchor":"top",
        "y":0.97,
        "xanchor":"left",
        "x":0.65
    })

    carbon_plot.show()

    pio.write_image(carbon_plot, file="lp_overhead.pdf", format='pdf')



def third_plot_adding_progress_counters():
    """second plot includes overhead for every start
    """
    prototype_fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)
    carbon_plot = make_carbon_trace_with_work_or_startup([(result_3["work"], 'Work'), (result_3["starting"], 'Starting'), (result_3["startup_finished"], 'Startup finished')])
    carbon_plot.update_layout(height=shared_height, width=shared_width)
    integers_plot = make_trace_from_variables([(result_3["startup_time_progressed"], 'Starting'), (result_3["work_time_progressed"], 'Work')])
    integers_plot.update_layout(height=shared_height, width=shared_width)


    prototype_fig.add_traces(integers_plot.data, rows=1, cols=1)
    prototype_fig.add_traces(carbon_plot.data, rows=2, cols=1)

    prototype_fig.update_layout({'yaxis1_title':'Progress', 'yaxis2_title': 'Carbon intensity in kgCO2/kWh', 'xaxis2_title':'Time slots'})
    add_common_styling_to_fig(prototype_fig)

    prototype_fig.update_layout(legend ={
        "yanchor":"top",
        "y":0.97,
        "xanchor":"left",
        "x":0.01
    })

    prototype_fig.update_layout(height=shared_height*2, width=shared_width)

    prototype_fig.show()

    pio.write_image(prototype_fig, file="lp_progress.pdf", format='pdf')

def get_color(event_str: str) -> str:
    if ('Start' in event_str):
        return 'rgba(255, 0, 0, 0.2)' 
    if ('Download' in event_str):
        return 'rgba(0, 255, 0, 0.2)'
    if ('High' in event_str):
        return 'rgba(236, 39, 245, 0.2)'
    if ('Low' in event_str or "eval" in event_str): # regrets
        return 'rgba(245, 172, 39, 0.2)'
    return 'rgba(0, 0, 0, 1)'

def get_label(event_str: str) -> str:
    if ('Start' in event_str):
        return 'Start Python' 
    if ('Download' in event_str):
        return 'Download data'
    if ('Start train' in event_str):
        return 'prepare dataset'
    if ('High' in event_str):
        return 'High power'
    if ('Low' in event_str):
        return 'Low power'
    return event_str

def plot_with_states():
    prototype_fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)
    carbon_plot = make_carbon_trace_with_work_or_startup([(result_4["work"], 'Work'), (result_4["starting"], 'Starting'), (result_4["startup_finished"], 'Startup finished')])
    
    integers_plot = make_trace_from_variables([(result_4["startup_time_progressed"], 'Starting'), (result_4["work_time_progressed"], 'Work')])

    phases_variables_with_name = [
        *[(value['variable'], key) for (key, value) in result_4["lin_function_dicts"]['startup'].items()],
        *[(value['variable'], key) for (key, value) in result_4["lin_function_dicts"]['work'].items()]
    ]

    prototype_fig.add_traces(integers_plot.data, rows=1, cols=1)
    prototype_fig.add_traces(carbon_plot.data, rows=2, cols=1)

    for variable, name in phases_variables_with_name:
        bools = [True if pulp.value(variable[t]) > 0 else False for t in range(len(variable))]
        groups = find_upper_lower_index_for_phases(bools)
        for start, end in groups:
            print(f"{start}, {end}, {name}")
            prototype_fig.add_shape(type='rect',
                name=name, layer="below",
                label=dict(text=get_label(name), textangle=-90),
                x0=start, y0=0, x1=end, y1=100,
                line=dict(color='rgba(0, 0, 0, 0)'),
                row=1, col=1, fillcolor=get_color(name)
            )

    prototype_fig.update_layout({'yaxis1_title':'Progress', 'yaxis2_title': 'Carbon intensity in kgCO2/kWh', 'xaxis2_title': 'Time slots'})
    
    add_common_styling_to_fig(prototype_fig)

    prototype_fig.update_layout(legend ={
        "yanchor":"top",
        "y":0.97,
        "xanchor":"left",
        "x":0.01
    })

    prototype_fig.show()

    pio.write_image(prototype_fig, file="lp_states.pdf", format='pdf')


first_plot_work_only()
second_plot_overhead_via_startup()
third_plot_adding_progress_counters()
plot_with_states()


72, 81, Start Python0
149, 158, Start Python0
82, 91, Download Data1
159, 168, Download Data1
92, 110, High2
169, 199, High2
200, 249, Low3
